<a href="https://colab.research.google.com/github/emmanuellfc/EM_Tufts_Fall2023/blob/main/GFDMDemonstration.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# @title Mount Google Drive.

# For getting files from drive.
from google.colab import drive
import os
drive.mount('/content/drive')
os.chdir('drive/My Drive/Colab Notebooks')

Mounted at /content/drive


In [2]:
# @title Dependency nonsense.
%%capture

# Package dependencies.
!apt install libcgal-dev libeigen3-dev
!pip install -U pygalmesh
!pip install networkx

import numpy as np
import matplotlib.pyplot as plt
from matplotlib.collections import LineCollection
from typing import Iterable
from typing import Callable
from typing import Optional
import math
import csv
from GeneralFiniteDifferences import MeshPoint
from GeneralFiniteDifferences import FiniteDifferences

In [ ]:
# @title Make a new boundary: A box with some irregular potentials.

# A series of points when strung together form a boundary.
def box_generator():
  for y in np.linspace(0,1,15,endpoint=False): yield (0,y) # Make the left edge.
  for x in np.linspace(0,1,15,endpoint=False): yield (x,1) # Make the top edge.
  for y in np.linspace(0,1,15,endpoint=False): yield (1,1-y) # Make the right edge.
  for x in np.linspace(0,1,15,endpoint=False): yield (1-x,0) # Make the bottom edge.

# Crude function that declares boundary conditions.
def box_boundary(pt:MeshPoint):
  if pt.pos[0] <= 0.01: return 2 - pt.pos[1]
  if pt.pos[0] >= 0.99: return 1 - pt.pos[1]
  if pt.pos[1] <= 0.01: return 2 - pt.pos[0]
  if pt.pos[1] >= 0.99: return 1 - pt.pos[0]

# Make a new plot that shows the bare boundary...
fig,ax = plt.subplots(dpi=100)
ax.set_aspect('equal')
plt.scatter([ pt[0]                       for pt in box_generator() ],
            [ pt[1]                       for pt in box_generator() ],
          c=[ box_boundary(MeshPoint(pt)) for pt in box_generator() ])
cbar = plt.colorbar()
cbar.set_label('Potential')
plt.show()

In [ ]:
# @title Fill the encolsure with points.

# Make a new solver object.
box_solution = FiniteDifferences()
# Inform it with some parameters.
box_solution.set_max_edge_size(0.15)
box_solution.set_num_neighbors(6)
box_solution.inform_with_boundary_conditions(box_boundary)
box_solution.make_points_from_boundary(box_generator())

fig,ax = plt.subplots(dpi=100)
ax.set_aspect('equal')
plt.scatter([ pt.pos[0]                           for pt in box_solution.all_points ],
            [ pt.pos[1]                           for pt in box_solution.all_points ],
          c=[ ( 0 if pt.pot is None else pt.pot ) for pt in box_solution.all_points ])
cbar = plt.colorbar()
cbar.set_label('Potential')
plt.show()


In [ ]:
# @title Make connections between points.

box_solution.make_graph_from_points()

# Draw the connectivity plot.
box_solution.draw_connectivity()


In [ ]:
# @title Sorcery! Solve for potential inside the region.

box_solution.make_systems_matrix()
box_solution.solve_systems_matrix()

fig,ax = plt.subplots(dpi=100)
ax.set_aspect('equal')
plt.scatter([ pt[0]  for pt in box_solution.all_points ],
            [ pt[1]  for pt in box_solution.all_points ],
          c=[ pt.pot for pt in box_solution.all_points ])
cbar = plt.colorbar()
cbar.set_label('Potential')
plt.show()
